# Actividad 1: El barbero durmiente

En este notebook vamos a explicar una propuesta de solución para el problema del "Barbero Durmiente". En este problema tenemos un barbero que espera clientes en su barbería. Mientras no hay clientes el barbero duerme. Cuando llega un cliente el barbero despierta y procede a atenderlo. Si cuando un cliente llega el barbero está ocupado pasa a esperar su turno en alguna de las tres sillas que dispone para sentarse. Si un cliente llega y las sillas están ocupadas esperará fuera hasta que se libere alguna.

Durante la implementación de esta solución se deben tener en cuenta varios aspectos importantes de la programación concurrente como son las condiciones de carrera o problemas de vivacidad de los hilos. Vamos a pasar a definir los mecanismos de sincronización utilizados para lograr nuestro objetivo.

- `self.espera_sala`: Se trata de un `Semaphore` inicializado con un entero igual al número de sillas que hay en la barbería. Con este semáforo controlamos el número máximo de hilos (clientes) que tenemos esperando de forma simutánea en las sillas de la barbería. Si un hilo queda bloqueado en el semáforo (debido a que ya está ocupado por tantos hilos como sillas) se entiende que espera "fuera" de la barbería.
- `self.exclusion_sala`: Se trata de un `Lock` que se utiliza para garantizar el acceso en exclusión mútua a variables compartidas entre todos los hilos. A continuación definiremos también estas variables compartidas. 
- `self.dormir`: Se trata de una `Condition` que utilizaremos para controlar si el barbero duerme o no. Si en la barbería no hay clientes esperando el barbero se duerme invocando `wait()` sobre esta condición. Cuando llega un cliente y se sienta, avisa al barbero con `notify()` para despertarlo y que comience a atender a los clientes.
- `self.sillon`: Se trata de una `Condition` que usaremos para avisar a los clientes que están esperando en las sillas de que el sillón se ha quedado vacío (se ha terminado de cortar el pelo a un cliente).

Por otra parte, en la implementación propuesta únicamente usamos una variable compartida entre los hilos. Exponemos el propósito de esta variable a continuación.

- `self.clientes`: Se trata de una `list` de hilos que almacena los clientes que están sentados en las sillas. Cómo sabemos, en programación concurrente existe el problema de la inanición de los hilos, esto es, un hilo que nunca consigue acceder a los recursos que necesita para proseguir su ejecución. Para evitar este problema hacemos uso de esta lista y almacenamos los clientes por orden de llegada. Cuando el barbero atiende a un cliente atiende al que más tiempo lleva esperando, es decir, el cliente que está primero en la lista. De esta forma, garantizamos que ningún hilo espere durante un tiempo indefinido (o excesivamente largo) y pase al sillón en algún momento. Imponemos así un orden FIFO de atención a los clientes.

De esto último que se ha explicado, es importante mencionar que la inanición no es un problema con un peso excesivamente significativo cuando a la barbería van a acceder pocos clientes (hilos), pues en el ejemplo que vamos a proponer, vamos a usar un barbero y seis clientes. Sin embargo, problemas en los que se usen un número más alto de clientes, o se generen clientes de forma indefinida se podría convertir en un problema real. Nos encontraríamos en una situación en la que el hilo nunca consigue acceder al sillón pero tampoco libera su asiento, ocupando de forma indefinida un recurso. Adjuntamos a continuación el código implementado. 

In [1]:
import threading
import time
import random

class Barberia:
    def __init__(self, num_sillas = 3):
        ### Inicialización de mecanismos y variables de control para clientes y acceso a la sala
        # 1.- Control de entrada a la sala
        # Para este mecanismo de control vamos a utilizar un Semaphore inicializado
        # con value = num_sillas. El parámetro num_sillas nos permite controlar el número de "sillas"
        # de las que disponemos, es decir, cuantos clientes pueden esperar dentro de la barbería
        self.espera_sala = threading.Semaphore(num_sillas)
        # 2.- Control del número de clientes en sala
        # Usaremos la lista clientes para almacenar los hilos que tenemos esperando dentro de la barbería
        # El objetivo de esta lista es imponer una política FIFO sobre los clientes esperando. En otras palabras,
        # el barbero SIEMPRE atenderá al cliente que lleva más tiempo esperando (el que entra antes) y, de esta forma,
        # evitamos problemas de inanición de hilos tan característicos de la programación concurrente.
        self.num_sillas = num_sillas
        self.clientes = []
        # 3.- Exclusión mútua para modificación de variables de control
        # Utilizaremos un Lock que cada hilo podrá adquirir para modificar la variable (lista)
        # clientes sin producir condiciones de carrera.
        self.exclusion_sala = threading.Lock()


        ### Incialización de mecanismos y variables de control para el barbero
        # 1 .- Control de acceso al sillón 
        # Lo inicializaremos como un Condition(). De esta forma, si el barbero está ocupado cortando el pelo
        # los hilos quedarán a la espera en esta condición. Cuando el barbero termine usaremos
        # un mecanismo de notificación para, en caso de que hayan clientes a la espera, pase alguno
        # de ellos. 
        self.sillon = threading.Condition()
        # 2.- Control para poner el barbero a dormir y ser despertado
        self.dormir = threading.Condition()


    def actividad_barbero(self):
        while True:
            # En primer lugar el barbero comprueba si hay clientes y debe despertar
            self.exclusion_sala.acquire()
            with self.dormir:
                if len(self.clientes) == 0:
                    print(f"{threading.current_thread().name} no tiene clientes y pasa a dormir.\n")
                    self.exclusion_sala.release()
                    self.dormir.wait()
                else:
                    print(f"{threading.current_thread().name} tiene trabajo\n")
                    self.exclusion_sala.release()
            # Si llegamos a esta región del código el barbero está despierto y debe comenzar
            # a atender a los clientes. Como explicábamos en el constructor, la idea es implementar
            # una política FIFO de atención a los clientes. Para ello, vamos a tomar la lista de clientes
            # y atender al cliente en la posición 0 de la lista. Eliminaremos ese elemento de la lista
            # para indicar que ya ha sido atendido.
            self.exclusion_sala.acquire()
            cliente_actual = self.clientes.pop(0)
            self.exclusion_sala.release()
            print(f"{threading.current_thread().name} pasa a atender a {cliente_actual.name}\n")
            time.sleep(5)
            print(f"{threading.current_thread().name} termina de atender a {cliente_actual.name}\n")
            # Una vez el cliente ha sido atendido, el barberó dará una notificación sobre sillon
            # para que 
            with self.sillon:
                self.sillon.notify_all()     
    
    def actividad_cliente(self):
        print(f"El cliente {threading.current_thread().name} llega a la barbería\n")
        # El cliente llega y comprueba si puede sentarse a esperar en alguna silla
        # Opción 1.- Todas las sillas están ocupadas y el cliente espera "fuera" de la barbería
        # Opción 2.- Hay sillas libres y el cliente se sienta a esperar
        self.espera_sala.acquire()
        # Si llegamos a esta región del código habían sillas libres y el cliente se sienta a esperar.
        # Para simbolizar esto añadimos el hilo actual a la lista de clientes.
        print(f"El cliente {threading.current_thread().name} se sienta a esperar su turno\n")
        self.exclusion_sala.acquire()
        self.clientes.append(threading.current_thread())
        self.exclusion_sala.release()
        # Notificamos al barbero para despertarlo en caso de que esté dormido
        with self.dormir:
            self.dormir.notify()
        # Una vez notificado, el barbero debería despertar y comenzar a atender a los clientes
        # Se atenderá por orden FIFO por lo que los clientes que están sentados deberán esperar 
        # a que sea su turno. Cuando el barbero libere el sillón se lanzará una notificación y
        # el hilo que haya sido atendido saldrá del bucle while para abandonar la barbería
        while threading.current_thread() in self.clientes:
            with self.sillon:
                self.sillon.wait()
        # El cliente que abandone la barbería liberará el Semaphore para que otros puedan entrar.
        self.espera_sala.release()
        print(f"El cliente {threading.current_thread().name} sale de la barbería\n")

En los comentarios del código adjunto se explican algunos aspectos importantes de la sincronización y los mecanismos utilizados. Sin embargo, se considera adecuado realizar una explicación general del protocolo establecido para resolver el problema. 

- **Barbero**: El barbero comienza comprobando (en exclusión mútua) si hay clientes esperando en las sillas comprobando la longitud de la lista `self.clientes`. Si no hay clientes esperando se pone a "dormir" invocando `wait()` sobre la condición `self.dormir`. En caso contrario comienza a trabajar. El barbero procede entonces a seleccionar (y eliminar de la lista `self.clientes`) al cliente que lleva más tiempo esperando. Este acceso se hace en exclusión mútua para evitar condiciones de carrera sobre `self.clientes`. Una vez seleccionado el cliente se realiza un `time.sleep(5)` para simular que el corte de pelo toma algo de tiempo. Una vez concluido, se realiza un `notify_all()` sobre `self.sillon` para notificar a los clientes de que se ha liberado el sillón. A continuación discutiremos la utilizad de esta notificación.
- **Cliente**: El cliente comienza llegando a la barbería e invocando `acquire()` sobre el semáforo `self.espera_sala`. En caso de que todas las sillas estén ocupadas el hilo quedará bloqueado a la espera de que se libere alguna (esto sería como esperar "fuera" de la barbería). Una vez consigue adquirir el semáforo, el cliente se sienta a esperar su turno y, en exclusión mútua, se añade a la lista `self.clientes`. A continuación avisa al barbero para que despierte invocando `notify()` sobre `self.dormir`. Una vez llegado a este punto, el cliente está a la espera de que el barbero lo atienda, para simular esta espera ponemos a cada hilo a esperar sobre la condición `self.sillon`. Cuando el barbero termina un servicio notifica a los hilos esperando sobre esta condición y todos ellos pasan a comprobar si todavía se encuentran en la lista `self.clientes`. Si un hilo no se encuentra en esa lista significa que el barbero ya lo ha atendido por lo que sale del bucle y notifica de su salida de la barbería. Ejecuta además un `release()` sobre `self.espera_sala` para permitir que entre algún cliente que estuviese esperando fuera.

In [ ]:
# Incializamos la barbería con 3 sillas para la espera
barberia = Barberia(3)
# Creamos el hilo que representa al barbero
barbero = threading.Thread(target = barberia.actividad_barbero, name = 'Barbero')
cli = []
# Creamos los hilos de los clientes
names = ['Harry Potter', 'Ragnar', 'Walter White', 'Voldemort', 'Frodo', 'Han Solo']
for name in names:
    cli.append(threading.Thread(target = barberia.actividad_cliente, name = name))
    
# Comenzamos los hilos dejando una pausa para que el barbero comience durmiendo
barbero.start()
time.sleep(1)
# Vamos a introducir cierto retardo en la llegada de los clientes para evitar que lleguen
# al mismo tiempo
for client in cli:
    client.start()
    time.sleep(random.randint(2, 8))
# Esperamos a que terminen todos los hilos
barbero.join()
for client in cli:
    client.join()

Barbero no tiene clientes y pasa a dormir.

El cliente Harry Potter llega a la barbería

El cliente Harry Potter se sienta a esperar su turno

Barbero pasa a atender a Harry Potter

Barbero termina de atender a Harry Potter

Barbero no tiene clientes y pasa a dormir.

El cliente Harry Potter sale de la barbería

El cliente Ragnar llega a la barbería

El cliente Ragnar se sienta a esperar su turno

Barbero pasa a atender a Ragnar

Barbero termina de atender a Ragnar

Barbero no tiene clientes y pasa a dormir.

El cliente Ragnar sale de la barbería

El cliente Walter White llega a la barbería

El cliente Walter White se sienta a esperar su turno

Barbero pasa a atender a Walter White

El cliente Voldemort llega a la barbería

El cliente Voldemort se sienta a esperar su turno

Barbero termina de atender a Walter White

Barbero tiene trabajo

Barbero pasa a atender a Voldemort

El cliente Walter White sale de la barbería

El cliente Voldemort sale de la barbería

El cliente Frodo llega a l

En la celda anterior mostramos un ejemplo de ejecución con un barbero y seis clientes. El objetivo evidentemente es que el barbero logre atender a todos los clientes y que ninguno quede sin atender. La ejecución puede lanzarse varias veces para obtener resultados diferentes, pues hemos introducido cierto retardo entre la llegada de los clientes (para simular que no llegan todos al mismo tiempo). 